# **NLP3_Run1**

In [1]:
!pip install openai
!pip install python-dotenv
!pip3 install pyomo
!apt install glpk-utils
!pip install glpk

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 325.5/325.5 kB 5.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 6.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 7.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 5.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 80.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.6/49.6 kB 5.6 MB/s eta 0:00:00
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following additional packages will be installed:
  libamd2 libcolamd2 libglpk40 libsuitesparseconfig5
Suggested packages:
  libiodbc2-dev
The following NEW packages will be installed:
  glpk-utils libamd2 libcolamd2 libglpk40 libsuitesparseconfig5
0 upgraded, 5 newly installed, 0 to remove and 45 not upgraded.
Need to get 625 kB of archives.
After this operation, 2,158 kB of additional disk space will be used.
Ge

In [2]:

import openai
import os
from IPython.display import Markdown
import pyomo


### **Accessing the GPT4 API**

In [4]:
import os
from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv('api_file.env'))
my_api_key = os.environ['api_key_env']



### **Generate Mathematical Model**

In [5]:
problem = """ A buyer needs to acquire 239,600,480 units of a product and is considering bids from five suppliers, labeled A through E, each of whom can only supply a portion of the total required amount.
Each vendor has proposed different pricing structures, incorporating both setup fees and variable unit costs that change based on the quantity ordered.

The buyer's objective is to allocate the order among these suppliers to minimize overall costs, accounting for both setup and unit costs.

Vendor A offers a set up cost of $3855.34 and a unit cost of $61.150 per thousand of units.
Vendor A can supply up to 33 million units.

Vendor B offers a set up cost of $125,804.84 if purchasing between 22,000,000-70,000,000 units from vendor B with a unit cost of $68.099 per thousand units.
If purchasing between 70,000,001-100,000,000 units from vendor B, the set up cost increases to $269304.84 and the unit cost sinks to $66.049 per thousand units.
If purchasing between 100,000,001-150,000,000 units from vendor B, the unit cost per thousand units further decreases to $64.099, but the set up cost increases to $464304.84.
If purchasing between 150,000,001 and 160,000,000 units from vendor B, the unit cost is $62.119 per thousand units and the set up cost equals $761304.84.

Vendor C offers set up costs of $13,456.00 and a unit cost of $62.019 per thousand units.
Vendor C can supply up to 165.6 million units. Vendor D offers set up costs of $6,583.98 and a unit cost of $72.488 for a set of thousand units.

Vendor D can supply up to 12 million units at a price of $72.488 per thousand units and with a set up cost of $6583.98.

Vendor E offers free set up if purchasing between 0 and 42 million units of vendor E with a unit price of $70.150 per thousand units.
If purchasing between 42,000,001 and 77 million units from vendor E, the unit cost starts at $68.150 per thousand units, but with every one million units purchased the price decreases at a rate of 0.05 percent. An additional set up cost of $84000 will be charged as well.

Note that zero units may be purchased from vendor B: otherwise no positive number of units less than 22,000,000 may be purchased. """

In [6]:
client = openai.OpenAI(api_key=os.environ['api_key_env'])

response1 = client.chat.completions.create(
    messages=[
        {"role": "system", "content": "Please formulate a mathematical optimization model for this problem. Include parameters, decision variables, the objective function and the constraints in your answer."},
        {
            "role": "user",
            "content": problem
    }
    ],
    model="gpt-4",
    seed = 1
)


In [7]:
print(response1.choices[0].message.content)

Model Formulation:

Parameters:
N = 239,600,480 units total needed
Vendor capacities = {A:33,000,000, B:160,000,000, C:165,600,000, D:12,000,000, E:77,000,000}
Vendor unit cost per thousand (in different intervals) and setup fees:

i. A: {3855.34, 61.150} 
ii. B: {125,804.84, 68.099; 269,304.84, 66.049; 464,304.84, 64.099; 761,304.84, 62.119}
iii. C: {13,456.00, 62.019}
iv. D: {6,583.98, 72.488}
v. E: {0, 70.150; 84,000, 68.150/1.0005^n}

Decision Variables:
For vendors A, C and D, xi units ordered from vendor i
For Vendor B, xi,j units ordered from vendor i in the jth price range
For Vendor E, xi,j units ordered from vendor i with jth unit cost.

Objective Function:
Minimize P = 3855.34 + 61.150xA/1000 
             + min([125,804.84 + 68.099*(sum(xB,j for all j))/1000 … 
    	               761,304.84 + 62.119*(sum(xB,j for all j))/1000])
             + 13,456.00 + 62.019*xC/1000 
             + 6,583.98 + 72.488*xD/1000
             + 0 + 70.150*xE1/1000
             + 84000 + 68.15

### **Generate Pyomo Code**

In [8]:

response2 = client.chat.completions.create(
    messages=[
        {"role": "system", "content": "Please write a python pyomo code for this optimization problem. Use sample data where needed. Indicate where you use sample data."},
        {
            "role": "user",
            "content": response1.choices[0].message.content
        }
    ],
    model="gpt-4",
    seed = 1
)

In [9]:
Markdown(response2.choices[0].message.content)

Here is a basic outline for formulating this problem in pyomo. Note that this is a non-linear, non-convex optimization problem with both integer and real variables. Hence, we will be using the pyomo.DAE solver.

```python
# required modules
from pyomo.environ import * 
from pyomo.dae import *

# Create a model
model = ConcreteModel()

# SETS
# vendors with single cost per unit
SINGLE_COST_VENDORS = ['A','C','D']

# vendors with multiple cost per unit
MULTIPLE_COST_VENDORS = ['B','E']

# Define parameters for single cost per unit vendors
VENDOR_CAPACITY = {'A' : 33000000, 'B' : 160000000, 'C' : 165600000, 'D' : 12000000 } 
VENDOR_COSTS = {'A' : 61.150, 'C' : 62.019, 'D' : 72.488} 

# Define parameters for multiple cost per unit vendors
# Note - These are dummy values. Replace them with your actual values
B_COST_RANGES = [125804.84,269304.84,464304.84]
B_COSTS = [68.099,66.049,64.099,62.119]
E_COST_RANGES = [84000,84000,84000]
E_COSTS = [70.150,68.150]

# VARIABLES
model.vendor_units = Var(SINGLE_COST_VENDORS, within=NonNegativeReals)
model.vendor_B_units = Var(len(B_COST_RANGES), within=NonNegativeReals)
model.vendor_E_units = Var(len(E_COST_RANGES), within=NonNegativeIntegers)

# OBJECTIVE
def objective_rule(model):
  cost = sum(model.vendor_units[i] * VENDOR_COSTS[i] for i in SINGLE_COST_VENDORS)
  cost += sum(model.vendor_B_units[i] * B_COSTS[i] for i in range(len(B_COST_RANGES)))
  cost += sum(model.vendor_E_units[i] * (E_COSTS[i]/(1.0005 ** sum(model.vendor_E_units[i])) for i in range(len(E_COST_RANGES))))

  return cost

model.objective = Objective(rule=objective_rule, sense=minimize)

# Constraint 1: Total purchase constraint: sum(xi for all i) = N
def total_purchase_rule(model):
    return sum(model.vendor_units[i] for i in SINGLE_COST_VENDORS) + sum(model.vendor_B_units[i] for i in range(len(B_COST_RANGES))) + sum(model.vendor_E_units[i] for i in range(len(E_COST_RANGES))) == N

model.total_purchase_cons = Constraint(rule=total_purchase_rule)

# Constraint 2: Vendor capacity constraint: xi <= vendor capacities{i} for all i
def vendor_capacity_rule(model, i):
    return model.vendor_units[i] <= VENDOR_CAPACITY[i]

model.vendor_capacity_cons = Constraint(SINGLE_COST_VENDORS, rule=vendor_capacity_rule)

# Constraint 3: No positive number of units less than 22,000,000 may be purchased from B: xB1 = 0 OR xB1 >= 22,000,000.
def min_units_B_rule(model):
    return model.vendor_B_units[0] >= 22000000

model.min_units_B_cons = Constraint(rule=min_units_B_rule)

# Constraint 4: Continuous purchase constraint for E: order from E must be in increments of 1 million units.
def increment_constraint_rule(model, i):
    return model.vendor_E_units[i] % 1000000 == 0

model.increment_cons = Constraint(range(len(E_COST_RANGES)), rule=increment_constraint_rule)

# create a solver
solver = SolverFactory('ipopt')

# solve
solver.solve(model, tee=True)
```

Please remember that I am using ipopt solver which might not be able to solve such a complex problem. Here I am assuming `N=239600480`.

### **Run the code Generated by GPT4**

In [10]:
# Install Python API for AMPL
!pip install amplpy --upgrade

from amplpy import AMPL,ampl_notebook

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 15.8 MB/s eta 0:00:00


In [11]:

_ = load_dotenv(find_dotenv('second_knitro_license.env'))
my_knitro_license = os.environ['knitro_license']

ampl = ampl_notebook(
    modules=["knitro"],  # modules to install
    license_uuid=my_knitro_license,  # license to use
)

Licensed to AMPL Community Edition License for <cevikmelis14@gmail.com>.


In [13]:
from pyomo.environ import *
from pyomo.dae import *

# Create a model
model = ConcreteModel()

# SETS
# vendors with single cost per unit
SINGLE_COST_VENDORS = ['A','C','D']

# vendors with multiple cost per unit
MULTIPLE_COST_VENDORS = ['B','E']

# Define parameters for single cost per unit vendors
VENDOR_CAPACITY = {'A' : 33000000, 'B' : 160000000, 'C' : 165600000, 'D' : 12000000 }
VENDOR_COSTS = {'A' : 61.150, 'C' : 62.019, 'D' : 72.488}

# Define parameters for multiple cost per unit vendors
# Note - These are dummy values. Replace them with your actual values
B_COST_RANGES = [125804.84,269304.84,464304.84]
B_COSTS = [68.099,66.049,64.099,62.119]
E_COST_RANGES = [84000,84000,84000]
E_COSTS = [70.150,68.150]

# VARIABLES
model.vendor_units = Var(SINGLE_COST_VENDORS, within=NonNegativeReals)
model.vendor_B_units = Var(len(B_COST_RANGES), within=NonNegativeReals)
model.vendor_E_units = Var(len(E_COST_RANGES), within=NonNegativeIntegers)

# OBJECTIVE
def objective_rule(model):
  cost = sum(model.vendor_units[i] * VENDOR_COSTS[i] for i in SINGLE_COST_VENDORS)
  cost += sum(model.vendor_B_units[i] * B_COSTS[i] for i in range(len(B_COST_RANGES)))
  cost += sum(model.vendor_E_units[i] * (E_COSTS[i]/(1.0005 ** sum(model.vendor_E_units[i])) for i in range(len(E_COST_RANGES))))

  return cost

model.objective = Objective(rule=objective_rule, sense=minimize)

# Constraint 1: Total purchase constraint: sum(xi for all i) = N
def total_purchase_rule(model):
    return sum(model.vendor_units[i] for i in SINGLE_COST_VENDORS) + sum(model.vendor_B_units[i] for i in range(len(B_COST_RANGES))) + sum(model.vendor_E_units[i] for i in range(len(E_COST_RANGES))) == N

model.total_purchase_cons = Constraint(rule=total_purchase_rule)

# Constraint 2: Vendor capacity constraint: xi <= vendor capacities{i} for all i
def vendor_capacity_rule(model, i):
    return model.vendor_units[i] <= VENDOR_CAPACITY[i]

model.vendor_capacity_cons = Constraint(SINGLE_COST_VENDORS, rule=vendor_capacity_rule)

# Constraint 3: No positive number of units less than 22,000,000 may be purchased from B: xB1 = 0 OR xB1 >= 22,000,000.
def min_units_B_rule(model):
    return model.vendor_B_units[0] >= 22000000

model.min_units_B_cons = Constraint(rule=min_units_B_rule)

# Constraint 4: Continuous purchase constraint for E: order from E must be in increments of 1 million units.
def increment_constraint_rule(model, i):
    return model.vendor_E_units[i] % 1000000 == 0

model.increment_cons = Constraint(range(len(E_COST_RANGES)), rule=increment_constraint_rule)

# create a solver
solver = SolverFactory('knitro')

# solve
solver.solve(model, tee=True)

TypeError: Cannot create a Set from data that does not support __contains__.  Expected set-like object supporting collections.abc.Collection interface, but received 'int'.

### **Edit and Run the code for the mathematical model produced by GPT4 (Circumstantial)**

In [27]:
from pyomo.environ import *
from pyomo.dae import *

# Create a model
model = ConcreteModel()

# SETS
# vendors with single cost per unit
SINGLE_COST_VENDORS = ['A','C','D']

# vendors with multiple cost per unit
MULTIPLE_COST_VENDORS = ['B','E']

# Define parameters for single cost per unit vendors
VENDOR_CAPACITY = {'A' : 33000000, 'B' : 160000000, 'C' : 165600000, 'D' : 12000000 }
VENDOR_COSTS = {'A' : 61.150, 'C' : 62.019, 'D' : 72.488}

# Define parameters for multiple cost per unit vendors
# Note - These are dummy values. Replace them with your actual values
B_COST_RANGES = [125804.84,269304.84,464304.84]
B_COSTS = [68.099,66.049,64.099,62.119]
E_COST_RANGES = [84000,84000,84000]
E_COSTS = [70.150,68.150]

# VARIABLES
model.vendor_units = Var(SINGLE_COST_VENDORS, within=NonNegativeReals)
model.vendor_B_units = Var(B_COST_RANGES, within=NonNegativeReals)
model.vendor_E_units = Var(E_COST_RANGES, within=NonNegativeIntegers)

# OBJECTIVE
def objective_rule(model):
  cost = sum(model.vendor_units[i] * VENDOR_COSTS[i] for i in SINGLE_COST_VENDORS)
  cost += sum(model.vendor_B_units[i] * B_COSTS[i] for i in range(len(B_COST_RANGES)))
  cost += sum(model.vendor_E_units[i] * (E_COSTS[i]/(1.0005 ** sum(model.vendor_E_units[i])) for i in range(E_COST_RANGES)))

  return cost

model.objective = Objective(rule=objective_rule, sense=minimize)

# Constraint 1: Total purchase constraint: sum(xi for all i) = N
def total_purchase_rule(model):
    return sum(model.vendor_units[i] for i in SINGLE_COST_VENDORS) + sum(model.vendor_B_units[i] for i in range(len(B_COST_RANGES))) + sum(model.vendor_E_units[i] for i in range(len(E_COST_RANGES))) == N

model.total_purchase_cons = Constraint(rule=total_purchase_rule)

# Constraint 2: Vendor capacity constraint: xi <= vendor capacities{i} for all i
def vendor_capacity_rule(model, i):
    return model.vendor_units[i] <= VENDOR_CAPACITY[i]

model.vendor_capacity_cons = Constraint(SINGLE_COST_VENDORS, rule=vendor_capacity_rule)

# Constraint 3: No positive number of units less than 22,000,000 may be purchased from B: xB1 = 0 OR xB1 >= 22,000,000.
def min_units_B_rule(model):
    return model.vendor_B_units[0] >= 22000000

model.min_units_B_cons = Constraint(rule=min_units_B_rule)

# Constraint 4: Continuous purchase constraint for E: order from E must be in increments of 1 million units.
def increment_constraint_rule(model, i):
    return model.vendor_E_units[i] % 1000000 == 0

model.increment_cons = Constraint(range(len(E_COST_RANGES)), rule=increment_constraint_rule)

# create a solver
solver = SolverFactory('knitro')

# solve
solver.solve(model, tee=True)

ERROR:pyomo.core:Rule failed when generating expression for Objective objective with index None:
KeyError: "Index '0' is not valid for indexed component 'vendor_B_units'"
ERROR:pyomo.core:Constructing component 'objective' from data=None failed:
    KeyError: "Index '0' is not valid for indexed component 'vendor_B_units'"


KeyError: "Index '0' is not valid for indexed component 'vendor_B_units'"